## Enunciado del problema

Se requiere construir con modulo donde se puedan consolidar diferentes hojas de varios archivos Excel y dejar en un solo documento. El reto esta en que las hojas no pueden perder su formato original y estos archivos se podrán guardar en carpetas según el año y mes de ejecución. Tener en cuenta que el nombre de los archivos debe ser dinámico, tanto del origen como del destino (el origen de los archivos insumos es diferente para cada uno de ellos y la ruta destino se especifica según la  decisión del usuario). NOTA: De ser necesario se puede implementar una tabla paramétrica para configurar el comportamiento del módulo.

## Solución 

Autor: Daniel Felipe Solano León

e-mail: dasolano@bancolombia.com.co

Celular: 3192031658

In [25]:
import xlwings as xw
import os
from datetime import date
import shutil
import openpyxl
import pathlib

current_filepath = pathlib.Path('main.ipynb').parent.resolve()

def get_filepaths(root_path):
  try:
    root_iterator = os.scandir(root_path)
    paths = []
    for entry in root_iterator:
      if entry.is_file():
        paths.append(entry.path)
      elif entry.is_dir():
        paths.extend(get_filepaths(entry.path))
    return paths
  except:
    return []

input_paths = ['folder1', 'folder2\inner21', 'another_folder']
output_dir_path = 'output'
output_filename = 'output_filename'

all_paths = []
for path in input_paths:
  all_paths.extend(get_filepaths(path))

output_dir_date = f'{output_dir_path}\{date.today().year}\{date.today().month}'

if os.path.exists(output_dir_date):
  shutil.rmtree(output_dir_date)
  os.makedirs(output_dir_date)
else:
  os.makedirs(output_dir_date)

wb = openpyxl.Workbook()
path = f'{output_dir_date}\{output_filename}.xlsx'
wb.save(path)

output_excel = xw.Book(f'{current_filepath}\{path}')

for path in all_paths:
  full_path = f'{current_filepath}\{path}'
  current_excel = xw.Book(full_path)

  for sheet_number in range(1, len(current_excel.sheets) + 1):
    sheet = current_excel.sheets(sheet_number)
    sheet.api.Copy(Before = output_excel.sheets(1).api)

output_excel.sheets[-1].delete()
output_excel.save()
output_excel.app.quit()